<a href="https://colab.research.google.com/github/rainfireliang/COMM6320_IntroductionR/blob/master/LLM_for_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Tutorial for Text Classification using OpenAI

Hai Liang, 2022/11/24

The Chinese University of Hong Kong

hailiang@cuhk.edu.hk

https://drhailiang.com/

## Preparation

Using pip to install OpenAI:

In [ ]:
pip install openai

Now, we need to create an OpenAI API key. Please follow the steps stated here: https://www.maisieai.com/help/how-to-get-an-openai-api-key-for-chatgpt. Using the following function to input the API key.

In [ ]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-xxx",
)

# an example of creating a completion:
completion = client.completions.create(model='curie', prompt="Hello, world!")

# these are methods to check the content of returned completions:
print(completion.choices[0].text)
print(dict(completion).get('usage'))
print(completion.model_dump_json(indent=2))

” >> circle.distribute(); call;

Connection management


CompletionUsage(completion_tokens=16, prompt_tokens=4, total_tokens=20)
{
  "id": "cmpl-8Sy6N2PqvV7j89TKiNPNwDW8XPXry",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\u201d >> circle.distribute(); call;\n\nConnection management\n\n"
    }
  ],
  "created": 1701914907,
  "model": "curie",
  "object": "text_completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 4,
    "total_tokens": 20
  }
}


## Zero-shot Text Classification

It means that we can classify the text without training the model. We use sentiment analysis of IMDB movie reviews as an example.

The basic logic is to "prompt" the ChatGPT to classify the text as positive or negative. The prompt text would be something similar to the below.

In [ ]:
content = """What is the sentiment expressed in the following IMDB movie review? Select sentiment value from positive or negative. \n
Return only the sentiment value. Movie review: {}""".format("the movie is great")

print(content)

What is the sentiment expressed in the following IMDB movie review? Select sentiment value from positive or negative. 

Return only the sentiment value. Movie review: the movie is great


Now, we can create a completion, using the GPT3.5 model. You can use "gpt-4" with higher accuracy but more expensive.

In [ ]:
sentiment = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="gpt-3.5-turbo", # gpt-4
    temperature=0,
)

# print the returned results
print(sentiment.model_dump_json(indent=2))

{
  "id": "chatcmpl-8Sy6OBjmhfgafIuLIw6sWrRxhPvH5",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "positive",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1701914908,
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 42,
    "total_tokens": 43
  }
}


The completion returned "positive": sentiment -> choices[0] -> message -> content.


In [ ]:
print (sentiment.choices[0].message.content)

positive


Put all together, we can create a function to find the sentiment:

In [ ]:
def find_sentiment(text):
    content = """What is the sentiment expressed in the following IMDB movie review?
    Select sentiment value from positive or negative.
    Return only the sentiment value. Movie review: {}""".format(text)
    sentiment = client.chat.completions.create(
        messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
    )
    return sentiment.choices[0].message.content

Let's test it out.

In [ ]:
print(find_sentiment("the movie is terrible"))
print(find_sentiment("the movie is great"))
print(find_sentiment("""I think I'm glad I did not read the book,
because what I watched was nothing short of wonderful in every "facet".
Mark Ruffalo and the sparsely seen Hugh Laurie were fantastic,
but still a notch or two below the magnificent Aria Mia Loberti in her premier performance.
And as a huge fan of the German series "Dark", I grinned wide when Louis Hoffman was on the screen.
Great screenplay (IMO), photography, music...it is all very memorable. With just four episodes, I imagine I will eventually watch again."""))

negative
positive
positive


More examples in Chinese

In [ ]:
print(find_sentiment("""A clean miss, one hopes Allied bombs landed truer than this melodramatic mush."""))
print(find_sentiment("""無敵爛片、無聊透頂"""))
print(find_sentiment("""一部略帶憂傷的影片，將人世間的苦楚敘述得入木三分"""))

negative
negative
positive


## Few-shot Text Classification
It means that we can classify the text with a few training examples.

In [ ]:
examples = [("A clean miss, one hopes Allied bombs landed truer than this melodramatic mush.", "positive"),
 ("無敵爛片、無聊透頂", "negative"),("一部略帶憂傷的影片，將人世間的苦楚敘述得入木三分", "positive")]


prompt_1 = """What is the sentiment expressed in the following IMDB movie review?
  Select sentiment value from positive or negative. \n
  Return only the sentiment value. And I will show a few examples with texts and
  the sentiment in a list of brackets:{}""".format(examples)

prompt_2 = """The Movie review is: {}""".format("the movie is great")

content = prompt_1 + "\n" + prompt_2

print(content)

What is the sentiment expressed in the following IMDB movie review?
  Select sentiment value from positive or negative. 

  Return only the sentiment value. And I will show a few examples with texts and
  the sentiment in a list of brackets:[('A clean miss, one hopes Allied bombs landed truer than this melodramatic mush.', 'positive'), ('無敵爛片、無聊透頂', 'negative'), ('一部略帶憂傷的影片，將人世間的苦楚敘述得入木三分', 'positive')]
The Movie review is: the movie is great


In [ ]:
def find_sentiment_few(text):
    content = prompt_1 + "\n" + """The Movie review is: {}""".format(text)
    sentiment = client.chat.completions.create(
        messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
    )
    return sentiment.choices[0].message.content

In [ ]:
print(find_sentiment_few("the movie is terrible"))
print(find_sentiment_few("一部略帶憂傷的影片，將人世間的苦楚敘述得入木三分"))
print(find_sentiment_few("這部戲劇是一部略帶憂傷的影片，將人世間的苦楚敘述得入木三分"))

negative
positive
positive


## Text Classification with Fine Tuning
It means that we can fine-tune the pre-trained GPT models to classify the text. Please check the offical webpage: https://platform.openai.com/docs/guides/fine-tuning.

Fine-tuning is currently available for the following models:

* gpt-3.5-turbo-1106 (recommended)
* gpt-3.5-turbo-0613
* babbage-002
* davinci-002
* gpt-4-0613 (experimental — eligible users will be presented with an option to request access in the fine-tuning UI)

In [ ]:
pip install scikit-llm

In [ ]:
from skllm.config import SKLLMConfig
SKLLMConfig.set_openai_key("sk-xxx")

In [ ]:
from skllm.datasets import get_classification_dataset
X, y = get_classification_dataset()

In [ ]:
y[:10]

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

In [ ]:
from skllm.models.gpt import GPTClassifier

clf = GPTClassifier(
        base_model = "gpt-3.5-turbo-0613",
        n_epochs = None, # int or None. When None, will be determined automatically by OpenAI
        default_label = "Random", # optional
)

clf.fit(X,y) # y_train is a list of labels
labels = clf.predict(X[:10])

Created new file. FILE_ID = file-SBMLkvrtMiOyUCpOY6lbG6G9
Waiting for file to be processed ...
Created new tuning job. JOB_ID = ftjob-2X52tW1Kxl1v9LySb3bbpNZr
[2023-12-07 02:10:21.883787] Waiting for tuning job to complete. Current status: validating_files
[2023-12-07 02:12:22.065627] Waiting for tuning job to complete. Current status: running
[2023-12-07 02:14:22.356142] Waiting for tuning job to complete. Current status: running
Finished training. Number of trained tokens: 19323.


100%|██████████| 10/10 [00:06<00:00,  1.58it/s]


In [ ]:
clf.predict(X)

100%|██████████| 30/30 [00:08<00:00,  3.49it/s]


['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral']